# Training Logic

In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [23]:
np.random.seed(7777)
tf.random.set_seed(7777)

In [24]:
class Cifar10DataLoader():
    def __init__(self):
        # data load
        (self.train_x, self.train_y), \
            (self.test_x, self.test_y) = tf.keras.datasets.cifar10.load_data()
        self.input_shape = self.train_x.shape[1:]

    def scale(self, x):

        return (x / 255.0).astype(np.float32)

    def preprocess_dataset(self, dataset):

        (feature, target) = dataset

        # scaling #
        scaled_x = np.array([self.scale(x) for x in feature])

        # label encoding #
        ohe_y = np.array([tf.keras.utils.to_categorical(
            y, num_classes=10) for y in target])
        
        return scaled_x, ohe_y.squeeze(1)

    def get_train_dataset(self):
        return self.preprocess_dataset((self.train_x, self.train_y))

    def get_test_dataset(self):
        return self.preprocess_dataset((self.test_x, self.test_y))

cifar10_loader = Cifar10DataLoader()
train_x, train_y = cifar10_loader.get_train_dataset()

print(train_x.shape, train_x.dtype)
print(train_y.shape, train_y.dtype)

test_x, test_y = cifar10_loader.get_test_dataset()

print(test_x.shape, test_x.dtype)
print(test_y.shape, test_y.dtype)

(50000, 32, 32, 3) float32
(50000, 10) float32
(10000, 32, 32, 3) float32
(10000, 10) float32


In [25]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Add

def build_resnet(input_shape):
    inputs = Input(input_shape)

    net = Conv2D(32, kernel_size=3, strides=2,
                 padding='same', activation='relu')(inputs)
    net = MaxPool2D()(net)
    
    net1 = Conv2D(64, kernel_size=1, padding='same', activation='relu')(net)
    net2 = Conv2D(64, kernel_size=3, padding='same', activation='relu')(net1)
    net3 = Conv2D(64, kernel_size=1, padding='same', activation='relu')(net2)
    
    net1_1 = Conv2D(64, kernel_size=1, padding='same')(net)
    net = Add()([net1_1, net3])
    
    net1 = Conv2D(64, kernel_size=1, padding='same', activation='relu')(net)
    net2 = Conv2D(64, kernel_size=3, padding='same', activation='relu')(net1)
    net3 = Conv2D(64, kernel_size=1, padding='same', activation='relu')(net2)
    
    net = Add()([net, net3])
    
    net = MaxPool2D()(net)
    
    net = Flatten()(net)
    net = Dense(10, activation="softmax")(net)

    model = tf.keras.Model(inputs=inputs, outputs=net, name='resnet')
    
    return model

model = build_resnet((32, 32, 3))
model.summary()

Model: "resnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_8 (Conv2D)              (None, 16, 16, 32)   896         ['input_2[0][0]']                
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 8, 8, 32)    0           ['conv2d_8[0][0]']               
                                                                                                  
 conv2d_9 (Conv2D)              (None, 8, 8, 64)     2112        ['max_pooling2d_2[0][0]']        
                                                                                             

## 학습하는 과정을 직접 만들어보자!

In [26]:
# 기존의 방법
# model.fit()

In [34]:
lr = 0.03
batch_size = 64     # 메모리 절약을 위해 2의 배수로 쓰는데, 실감상 체감은 되지 않는다고 한다. // 관례상!

In [35]:
opt=tf.keras.optimizers.Adam(lr)
loss_fn = tf.keras.losses.categorical_crossentropy
train_loss = tf.keras.metrics.Mean()
train_acc = tf.keras.metrics.CategoricalAccuracy()

In [36]:
# loss_fn([1, 0], [0.8, 0.2])
#         #정답

In [37]:
def train_step(x,y):
    with tf.GradientTape() as tape:
        pred = model(x)
        loss = loss_fn(y, pred)
                        # 정답치 , 예측치
    gradients = tape.gradient(loss, model.trainable_variables)
    # w.assign(w - lr*grad)
    opt.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(y)           # 결과값을 계속 누적시킴, 각 epoch 마다의 변화값을 보기 위해서는
    train_acc(y, pred)  

In [38]:
# for i in range(train_x.shape[0]):
#     print(i)

In [ ]:
# for e in epochs:
#     for batch_x, batch_y in dataset:
        # pred = model(batch_x)    # 예측치를 구하기
#         loss_fn(batch_y, pred)
#         gradients
#         weight_update
#         print()

In [43]:
for epoch in range(1):
    for i in range(train_x.shape[0] // batch_size):
        idx = i * batch_size
        x, y = train_x[idx : idx+batch_size], train_y[idx : idx + batch_size]
        train_step(x, y)
        print('{} / {}'.format(i, train_x.shape[0] // batch_size), end='\r')
                                                                    # print 값이 덮어쓰기된다.

    fmt = 'epoch : {} loss : {} acc : {}'
    print(fmt.format(
        epoch+1,
        train_loss.result(),        # 결과값을 계속 누적시킴, 각 epoch 마다의 변화값을 보기 위해서는
        train_acc.result()
    ))
    train_loss.reset_states()       # 리셋하면 된다.
    train_acc.reset_states()

epoch : 1 loss : 0.10000000149011612 acc : 0.1001216396689415


In [44]:
@tf.function        # 데코레이션, 선언됐을 때, 미리 구현해놓는다. // 속도가 빨라진다. (gpu를 쓸 때 효과적)
                    # 연산이 이뤄지는 함수를 따로 만들어 데코레이션 해주는게 기본이다.
def train_step(x,y):
    with tf.GradientTape() as tape:
        pred = model(x)
        loss = loss_fn(y, pred)
                        # 정답치 , 예측치
    gradients = tape.gradient(loss, model.trainable_variables)
    # w.assign(w - lr*grad)
    opt.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(y)           # 결과값을 계속 누적시킴, 각 epoch 마다의 변화값을 보기 위해서는
    train_acc(y, pred)  

In [45]:
for epoch in range(1):
    for i in range(train_x.shape[0] // batch_size):
        idx = i * batch_size
        x, y = train_x[idx : idx+batch_size], train_y[idx : idx + batch_size]
        train_step(x, y)
        print('{} / {}'.format(i, train_x.shape[0] // batch_size), end='\r')
                                                                    # print 값이 덮어쓰기된다.

    fmt = 'epoch : {} loss : {} acc : {}'
    print(fmt.format(
        epoch+1,
        train_loss.result(),        # 결과값을 계속 누적시킴, 각 epoch 마다의 변화값을 보기 위해서는
        train_acc.result()
    ))
    train_loss.reset_states()       # 리셋하면 된다.
    train_acc.reset_states()

epoch : 1 loss : 0.10000000149011612 acc : 0.09865157306194305


In [ ]:
# model.compile(optimizer=, loss=, metrics=)